In [26]:
import os, glob, math, random
import numpy as np
from PIL import Image
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
import trimesh
import open3d as o3d
from open3d_utils import make_pcd, make_line_set, make_mesh
# export DISPLAY for remote visualization
# os.environ['DISPLAY'] = ":99"

In [21]:
def transfer_texture(output_dir, target_object_idx, source_mesh_data):
    # load target mesh
    target_mesh = o3d.io.read_triangle_mesh(os.path.join(output_dir, '{}_mesh.ply'.format(target_object_idx)))        
    target_vertices = np.asarray(target_mesh.vertices)
    target_faces = np.asarray(target_mesh.triangles)
    target_reconstructed_pointcloud = np.load(os.path.join(output_dir, '{}_point_cloud_pred.npy'.format(target_object_idx)))
    target_reconstructed_canonical_pointcloud = np.load(os.path.join(output_dir, '{}_point_cloud_pred_canonical.npy'.format(target_object_idx)))

    
    # transfer color from source mesh vertices --> source mesh 3D point cloud and source mesh canonical 3D point cloud
    source_vertices_kdtree = KDTree(source_mesh_data["source_vertices"])
    source_reconstructed_pointcloud_mapping_idx = source_vertices_kdtree.query(source_mesh_data["source_reconstructed_pointcloud"], k=1)[1]
    source_reconstructed_pointcloud_color = source_mesh_data["source_colors"][source_reconstructed_pointcloud_mapping_idx]
    
    # map target mesh vertices --> corresponding canonical 3D point cloud.
    target_reconstructed_pointcloud_kdtree = KDTree(target_reconstructed_pointcloud)
    target_vertices_mapping_idx = target_reconstructed_pointcloud_kdtree.query(target_vertices, k=1)[1]
    target_vertices_canonical_point = target_reconstructed_canonical_pointcloud[target_vertices_mapping_idx]
      
    # transfer color from source point cloud to target point cloud
    source_canonical_kdtree = KDTree(source_mesh_data["source_reconstructed_canonical_pointcloud"])
    target_vertices_canonical_mapping_idx = source_canonical_kdtree.query(target_vertices_canonical_point, k=1)[1]
    target_vertices_color = source_reconstructed_pointcloud_color[target_vertices_canonical_mapping_idx]
    
    # visualize target input image
    plt.imshow(np.asarray(Image.open(os.path.join(output_dir, '{}_image_input.png'.format(target_object_idx)))))
    plt.show()
    # visualize colored source mesh
    o3d.visualization.draw_geometries([make_mesh(source_mesh_data["source_vertices"], source_mesh_data["source_faces"], source_mesh_data["source_colors"][:,:3])])
    # visualize uncolored target mesh
    o3d.visualization.draw_geometries([make_mesh(target_vertices, target_faces)])
    # visualize colored target mesh
    o3d.visualization.draw_geometries([make_mesh(target_vertices, target_faces,target_vertices_color[:,:3])])

In [ ]:
# two steps to trasnfer texture: set the `output_dir` and set `texture_mesh_dir`

def load_source_mesh(textured_mesh_dir):
    # load source textured mesh (source mesh is first reconstructed by TARS, then painted using meshlab)
    source_mesh = o3d.io.read_triangle_mesh(os.path.join(textured_mesh_dir, 'textured_mesh.ply'))
    source_vertices = np.asarray(source_mesh.vertices)
    source_faces = np.asarray(source_mesh.triangles)
    source_colors = np.asarray(source_mesh.vertex_colors)
    source_colors = source_colors / 1.2
    source_reconstructed_pointcloud = np.load(os.path.join(textured_mesh_dir, 'point_cloud_pred.npy'))
    source_reconstructed_canonical_pointcloud = np.load(os.path.join(textured_mesh_dir, 'point_cloud_pred_canonical.npy'))
    source_mesh_data = {}
    source_mesh_data["source_vertices"] = source_vertices
    source_mesh_data["source_faces"] = source_faces
    source_mesh_data["source_colors"] = source_colors
    source_mesh_data["source_reconstructed_pointcloud"] = source_reconstructed_pointcloud
    source_mesh_data["source_reconstructed_canonical_pointcloud"] = source_reconstructed_canonical_pointcloud
    return source_mesh_data


def main():
    # need a display connect for open3d visualization
    # on open3d windown, press Ctrl + B to undo backface culling.
    
    # set output_dir to the data dump, and also provide the save directory.
    # path to output dump (generated by running evaluate.sh) eg: output/pascal3d/reconstruction_stage_two/chair/dump/
    output_dir = './'

    input_image_files = sorted(glob.glob(os.path.join(output_dir, '*_image_input.png')))
    colored_mesh_ply_files = sorted(glob.glob(os.path.join(output_dir, '*_color_coded_mesh.ply')))
    non_colored_mesh_ply_files = sorted(glob.glob(os.path.join(output_dir, '*[0-9]_mesh.ply')))

    # Please follow the instructions in "" to texture new source meshes yourself and then transfer to other meshes.
    # To use the one of the shared textured mesh as the source mesh, run the following:
    # texture_mesh_dir = <path_to_dumped_textured_meshes>
    # texture_mesh_dir = '../textured_meshes/70_mesh/'
    # texture_mesh_dir = '../textured_meshes/288_mesh/'
    texture_mesh_dir = '../textured_meshes/448_mesh/'
    source_mesh_data = load_source_mesh(texture_mesh_dir)

    # to transfer source mesh texture to all other meshes
    for img_path in input_image_files:
        target_idx = int(img_path.split('/')[-1].split('_')[0])
        transfer_texture(output_dir, target_idx, source_mesh_data)
        

main()